In [1]:
from pypdf import PdfReader
from openai import OpenAI
import os
from dotenv import load_dotenv
import numpy as np

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

PDF 파일에서 데이터를 읽고 분할

In [2]:
reader = PdfReader("data/spiderman1.pdf")
chunks = []
chunk_length = 1000

In [3]:
for page in reader.pages:
    text_page = page.extract_text()
    chunks.extend([text_page[i: i+chunk_length] for i in range(0, len(text_page), chunk_length)])

In [4]:
chunks[0]

'최근 변경 최근 토론\n어메이징  스파이더맨  실사영화  시리즈의 등장인물\n스파이더맨\nSpider -Man스파이더맨 ( 어메이징  스파이더맨  실사영화  시리\n즈)\n최근 수정  시각 : 2024-06-23 22:49:30\n\xa0 스파이더맨  트릴로지의  스파이더맨에  대한  내용은  스파이더맨 ( 스파이더맨  트릴로지 ) 문서를, 마블  시\n네마틱  유니버스의  스파이더맨에  대한  내용은  스파이더맨 ( 마블  시네마틱  유니버스 ) 문서를 참고하십시오 .\n스파이더맨  관련  틀\n[ 펼치기  · 접기  ]특수 기능\n여기에서  검색\n편집 요청 토론 역사\n분류: 피터 파커어메이징  스파이더맨  실사영화  시리즈 / 등장인물스파이더맨 ( 마블  시네마틱  유니버스 )…\n마블시네마틱 유니버 평행세계의 인물파이더맨 웨이 홈 장인물더 보기\n29\n'

PDF 파일에서 읽은 데이터 임베딩 생성

In [5]:
response = client.embeddings.create(model="text-embedding-ada-002", input=chunks)

In [6]:
result = [
    {
        "id": value.index,
        "vector": value.embedding,
        "text": chunks[value.index]
    } for value in response.data
]

임베딩을 faiss 인덱스로 저장

In [7]:
import faiss
import numpy as np
import pickle

embeddings = np.array([item['vector'] for item in result]).astype('float32')
ids = [item['id'] for item in result]
texts = {item['id']: item['text'] for item in result}

d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)

index.add(embeddings)

faiss.write_index(index, "faiss_index.bin")
with open("texts.pkl", "wb") as f:
    pickle.dump(texts, f)

랭체인 RAG agent 생성

In [10]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms.openai import OpenAI
from langchain.chains import RetrievalQA, StuffDocumentsChain
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

# faiss index 로드
index = faiss.read_index("faiss_index.bin")
with open("texts.pkl", "rb") as f:
    texts = pickle.load(f)


# faiss 기반 벡터 스토어 생성
class MyFAISS(FAISS):
    def __init__(self, index, texts):
        self.index = index
        self.texts = texts
    
    def similarity_search(self, query, k=4):
        query_embedding = np.array(query).astype('float32')
        D, I = self.index.search(np.array([query_embedding]), k)
        return [self.texts[i] for i in I[0]]

vector_store = MyFAISS(index, texts)
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

# 언어 모델 생성
llm = OpenAI(model="text-davinci-003")

# prompt template 생성
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="Context: {context}\n\nQuestion: {question}\n\nAnswer:"
)

# combine document chain 생성
combine_documents_chain = StuffDocumentsChain(
    llm=llm,
    prompt_template=prompt_template
)

# QA 체인 생성
qa_chain = RetrievalQA(
    retirever=retriever,
    combine_documents_chain=combine_documents_chain
)

AttributeError: 'MyFAISS' object has no attribute 'embedding_function'